## Data ETL Notebook

**Layer**: Silver

**Domain**: Risk-free

**Action**: Enrich NZDM Amounts - Merge series data on based on maturity date and series

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.riskfree_silver.002_nzdm_govtbonds_onissue_enriched AS

WITH cleaned_series_definitions AS (
  SELECT 
    *,
    -- Parse full date only if it looks like '20 September 2025'
    CASE 
      WHEN regexp_like(series, '^[0-9]{1,2} [A-Za-z]+ [0-9]{4}$') 
      THEN TO_DATE(series, 'd MMMM yyyy')
      ELSE NULL 
    END AS parsed_full_date,

    -- Fallback: parse 'May 2054' style partial dates → '15 May 2054'
    CASE 
      WHEN regexp_like(series, '^[A-Za-z]+ [0-9]{4}$') 
      THEN TO_DATE(CONCAT('15 ', series), 'd MMMM yyyy')
      ELSE NULL 
    END AS parsed_month_year
  FROM workspace.riskfree_metadata.series_definitions
),

normalized_series_definitions AS (
  SELECT 
    *,
    COALESCE(parsed_full_date, parsed_month_year) AS normalized_maturity
  FROM cleaned_series_definitions
  WHERE COALESCE(parsed_full_date, parsed_month_year) IS NOT NULL
),

joined_data AS (
  SELECT 
    nsd.series_id,
    nsd.group,
    govb.date_first_issued,
    govb.loan_prefix,
    govb.coupon,
    govb.maturity,
    govb.total_amt_outstanding_incl_rb_eqc_sresl_m,
    govb.rbnz_m,
    govb.eqc_m,
    govb.sresl_m
  FROM normalized_series_definitions nsd
  RIGHT JOIN riskfree_silver.001_nzdm_govtbonds_onissue_cleaned govb
    ON TRUNC(nsd.normalized_maturity, 'MM') = TRUNC(govb.maturity, 'MM')
)

SELECT * FROM joined_data;

-- Confirmation message
SELECT '✅ Silver table created: workspace.riskfree_silver.002_nzdm_govtbonds_onissue_enriched' AS message;